In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '16cb8e70-55da-4090-81ab-3c2d3e38b43c',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [ ]:
df = data

In [ ]:
# normalizes data makes it pretty in a dataframe & adds a new column timestamp
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
#if the file is not present create a file with dataframe df
if not os.path.isfile(r"C:\Users\kakku\OneDrive\Documents\Kaksha's Data\Python API Project\API.csv"):
    df.to_csv(r"C:\Users\kakku\OneDrive\Documents\Kaksha's Data\Python API Project\API.csv",header='column_names')
else:
    #print('Error')
    df.to_csv(r"C:\Users\kakku\OneDrive\Documents\Kaksha's Data\Python API Project\API.csv",mode='a',header=False) #if the file is already present then append the data

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start':'1',
        'limit':'5000',
        'convert':'USD'
         }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '16cb8e70-55da-4090-81ab-3c2d3e38b43c',
        }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
#         print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df
    
    if not os.path.isfile(r'C:\Users\kakku\OneDrive\Documents\API.csv'):
        df.to_csv(r'C:\Users\kakku\OneDrive\Documents\API.csv',header='column_names')
    else:
        df.to_csv(r'C:\Users\kakku\OneDrive\Documents\API.csv',mode='a',header=False)
    
   
    
#note: we get an error for data rate limit so follow the below steps
# Go to Anaconda prompt put in the error "jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10"

In [ ]:
import os
from time import time
from time import sleep

for i in range (333):
    api_runner()
    print('API Runner completed')
    sleep(60) #sleep for 1 minute
exit()

In [ ]:
df72 = pd.read_csv(r'C:\Users\kakku\OneDrive\Documents\API.csv')

In [ ]:
df72

In [ ]:
pd.set_option('display.float_format', lambda x : '%.5f' % x)

In [ ]:
df

In [ ]:
#grouping by names to take avg of different columns

df3 = df.groupby('name',sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()

In [ ]:
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
df4.head(40)

In [ ]:
#type(df3)  #Dataframe
type(df4)  #series

In [ ]:
df5 = df4.to_frame(name = 'values')
df5

In [ ]:
type(df5)

In [ ]:
df5.count()

In [ ]:
index = pd.Index(range(29808))

df6 = df5.reset_index()
df6

In [ ]:
df7 = df6.rename(columns = {'level_1':'percentage_change'})
df7

In [ ]:
df8 = df7.head(40)
df8

In [ ]:
df8['percentage_change'] = df8['percentage_change'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df8

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percentage_change',y='values',hue='name',data=df8,kind='point')

In [ ]:
df9 = df8[['name','percentage_change','values']]
df9 = df9.query("name == 'Bitcoin'")
df9

In [ ]:
sns.set_theme(style='darkgrid')

sns.lineplot(x='percentage_change',y='values',data=df9)